# Importações

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import zipfile

In [2]:
def load_data_year(year, cols):
    file = f'../../../microdados_anos/MICRODADOS_ENEM_{year}.csv'
    
    try:
        df = pd.read_csv(file, encoding='cp1252', sep=';', usecols=cols, nrows=1000)
    except:
        try:
            df = pd.read_csv(file, encoding='cp1252', usecols=cols, nrows=1000)
        except:
            df = pd.read_csv(file, encoding='cp1252', nrows=1000)
            df = df.rename(columns={'NOTA_LC': 'NU_NOTA_LC', 'NOTA_CN': 'NU_NOTA_CN', 'NOTA_CH': 'NU_NOTA_CH',  
                                    'NOTA_MT': 'NU_NOTA_MT','IN_PRESENCA_MT': 'TP_PRESENCA_MT', 'IDADE': 'NU_IDADE',
                                    'IN_PRESENCA_LC': 'TP_PRESENCA_LC', 'IN_PRESENCA_CH': 'TP_PRESENCA_CH', 
                                    'UF_RESIDENCIA': 'SG_UF_RESIDENCIA',  'ANO_CONCLUIU': 'TP_ANO_CONCLUIU', 
                                     'ST_CONCLUSAO': 'TP_ST_CONCLUSAO', 'IN_PRESENCA_CN': 'TP_PRESENCA_CN'})[cols]
    
    return df

In [3]:
cols_used = ['NU_INSCRICAO', 'NU_ANO', 'NO_MUNICIPIO_RESIDENCIA', 'SG_UF_RESIDENCIA',
             'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
             'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA',
             'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
             'TP_PRESENCA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
             'NU_NOTA_MT', 'NU_NOTA_COMP1',
             'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5',
             'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
             'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
             'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
             'Q023', 'Q024', 'Q025']

In [4]:
years = [2015, 2016, 2017, 2018, 2019]

In [5]:
def union_datas(years, cols):
    df_final = pd.DataFrame([], columns=cols)
    
    for y in years:
        df_final = pd.concat([df_final, load_data_year(y, cols)])
    
    return df_final

In [6]:
%%time

df_union = union_datas(years, cols_used)

Wall time: 120 ms


In [7]:
df_union.head()

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,150000000376,2015,Gravataí,RS,42,M,1.0,1,1,10,...,B,A,A,B,A,A,B,B,B,B
1,150000003053,2015,Montes Claros,MG,21,M,0.0,3,1,5,...,A,A,A,B,B,A,D,B,B,B
2,150000001161,2015,Olinda,PE,22,M,1.0,2,4,0,...,A,A,A,A,A,A,C,B,B,A
3,150000001335,2015,Belo Horizonte,MG,23,F,0.0,4,1,6,...,B,A,A,C,A,B,D,B,B,B
4,150000001219,2015,Teófilo Otoni,MG,18,M,0.0,3,1,2,...,A,A,B,B,A,A,C,B,B,B


# Pre-processing

In [8]:
pd.DataFrame({'types': df_union.dtypes, 'missing': df_union.isna().sum()})

,types,missing
NU_INSCRICAO,object,0
NU_ANO,object,0
NO_MUNICIPIO_RESIDENCIA,object,0
SG_UF_RESIDENCIA,object,0
NU_IDADE,object,0
TP_SEXO,object,0
TP_ESTADO_CIVIL,float64,177
TP_COR_RACA,object,0
TP_ST_CONCLUSAO,object,0
TP_ANO_CONCLUIU,object,0


In [40]:
df_final = df_union

In [41]:
#lambda x: pd.Series.mode(x) if (len(pd.Series.mode(x)) <= 1) else pd.Series.mode(x)[0]
def agg_lambda(x):
    mode = pd.Series.mode(x)
   
    if (len(mode) <= 1):
        return mode
    else:
        return mode[0]

aggregations = {
    'TP_ESTADO_CIVIL': agg_lambda,
    'NU_NOTA_CN':'mean',
    'NU_NOTA_CH': 'mean',
    'NU_NOTA_LC': 'mean',
    'NU_NOTA_MT': 'mean',
    'NU_NOTA_COMP1': agg_lambda,
    'NU_NOTA_COMP2': agg_lambda,
    'NU_NOTA_COMP3': agg_lambda,
    'NU_NOTA_COMP4': agg_lambda,
    'NU_NOTA_COMP5': agg_lambda,
    'NU_NOTA_REDACAO': agg_lambda
}
df_groups = df_final.groupby(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).agg(aggregations)
df_groups = df_groups.rename_axis(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).reset_index()

In [42]:
df_groups.head()

,TP_ESCOLA,TP_ST_CONCLUSAO,SG_UF_RESIDENCIA,TP_ESTADO_CIVIL,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO
0,1,1,AC,0.0,519.785714,591.357143,545.083333,538.383333,140,140,120.0,120.0,80.0,380.0
1,1,1,AL,0.0,490.832500,550.546341,515.032500,517.546154,120,120,120,120,80,520
2,1,1,AM,0.0,483.160000,553.093939,520.466667,501.696667,120,120,120,120,80,520
3,1,1,AP,0.0,461.900000,506.920000,508.720000,481.976923,120.0,120.0,120.0,120.0,0,0
4,1,1,BA,0.0,497.350198,543.312261,525.167692,499.225000,120,120,120,120,80,580


In [12]:
#imp_ec = SimpleImputer(strategy="most_frequent")
#imp_ec.fit(df_final[['TP_ESTADO_CIVIL']])

In [13]:
#df_final[['TP_ESTADO_CIVIL']] = imp_ec.transform(df_final[['TP_ESTADO_CIVIL']])

In [14]:
#df_groups = df_final.groupby(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).mean()
#df_groups = df_groups.rename_axis(['TP_ESCOLA', 'TP_ST_CONCLUSAO', 'SG_UF_RESIDENCIA']).reset_index()

In [15]:
#df_groups.head()

In [43]:
def add_null_values(x):
    if(sum(pd.isna(x)) > 0):
        df_x = df_groups.query('TP_ESCOLA==@x.TP_ESCOLA & TP_ST_CONCLUSAO==@x.TP_ST_CONCLUSAO & SG_UF_RESIDENCIA==@x.SG_UF_RESIDENCIA')
    
        for c in df_x.columns:
            if(pd.isna(x[c]) > 0):
                x[c] = df_x[c].values[0]
    return x

In [44]:
df_final = df_final.apply(add_null_values, axis=1)
df_final.dropna(inplace=True) # Perguntas do questionario com resposta nula

In [45]:
pd.DataFrame({'types': df_final.dtypes, 'missing': df_final.isna().sum()})

,types,missing
NU_INSCRICAO,int64,0
NU_ANO,int64,0
NO_MUNICIPIO_RESIDENCIA,object,0
SG_UF_RESIDENCIA,object,0
NU_IDADE,int64,0
TP_SEXO,object,0
TP_ESTADO_CIVIL,object,0
TP_COR_RACA,int64,0
TP_ST_CONCLUSAO,int64,0
TP_ANO_CONCLUIU,int64,0


# Incluindo Status Socioeconômico

## Incluindo a classe social do participante

### Respostas    
    - A	Nenhuma renda: 0.
    - B	Até  998,00:  Até 1.
    - C	De R$ 998,01 até R$ 1.497,00: 1 até 1,5.
    - D	De R$ 1.497,01 até R$ 1.996,00: 1,5 até 2.
    - E	De R$ 1.996,01 até R$ 2.495,00: 2 até 2,5.
    - F	De R$ 2.495,01 até R$ 2.994,00: 2,5 até 3.
    - G	De R$ 2.994,01 até R$ 3.992,00: 3 até 4.
    - H	De R$ 3.992,01 até R$ 4.990,00: 4 até 5.
    - I	De R$ 4.990,01 até R$ 5.988,00: 5 até 6.
    - J	De R$ 5.988,01 até R$ 6.986,00: 6 até 7.
    - K	De R$ 6.986,01 até R$ 7.984,00: 7 até 8.
    - L	De R$ 7.984,01 até R$ 8.982,00: 8 até 9.
    - M	De R$ 8.982,01 até R$ 9.980,00: 9 até 10.
    - N	De R$ 9.980,01 até R$ 11.976,00: 10 até 12.
    - O	De R$ 11.976,01 até R$ 14.970,00: 12 até 15.
    - P	De R$ 14.970,01 até R$ 19.960,00: 15 até 20.
    - Q	Mais de R$ 19.960,00: mais de 20.
    
    De 5000 participantes apenas 38 responderam como Q, então como seria inviável estimar a renda per capita consideramos que pertencem a classe A.


In [175]:
def add_class(x):
    income_dict = {'A': 0, 'B': 1, 'C': 1.5, 'D': 2, 'E': 2.5, 'F': 3, 'G': 4, 'H': 5, 'I': 6, 
                   'J': 7, 'K': 8, 'L': 9, 'M': 10, 'N': 12, 'O': 15, 'P': 20}
    if(x['Q006'] == 'Q'):
        x['TP_CLASSE_SOCIAL'] = 'A'
    else:
        per_capita = income_dict[x['Q006']]/x['Q005']

        if (per_capita <= 1): x['TP_CLASSE_SOCIAL'] = 'E'
        elif (per_capita <= 4):  x['TP_CLASSE_SOCIAL'] = 'D'
        elif (per_capita <= 10): x['TP_CLASSE_SOCIAL'] = 'C'
        else: x['TP_CLASSE_SOCIAL'] = 'B'
    
    return x

In [176]:
df_final = df_final.apply(add_class, axis=1)

In [177]:
df_final['TP_CLASSE_SOCIAL'].value_counts()

E    4083
D     816
C      53
A      38
Name: TP_CLASSE_SOCIAL, dtype: int64

In [46]:
%%time

# Save
file = '../datasets/integrated_data.csv'
df_final.to_csv(file, index=False)

#z = zipfile.ZipFile('../datasets/integrated_data.zip', 'w', zipfile.ZIP_DEFLATED)
#z.write(file)    
#z.close()

Wall time: 66.4 ms


# Testes anos

In [27]:
def load_data_year_2015(year, cols):
    file = f'../../../microdados_anos/MICRODADOS_ENEM_{year}.csv'
    try:
        df = pd.read_csv(file, encoding='cp1252', sep=';', usecols=cols, nrows=1000)
    except:
        df = pd.read_csv(file, encoding='cp1252', usecols=cols, nrows=1000)
    
    return df

In [28]:
df_2014 = pd.read_csv('../../../microdados_anos/MICRODADOS_ENEM_2014.csv', encoding='cp1252', nrows=1000)
df_2014.columns.values

array(['NU_INSCRICAO', 'NU_ANO', 'COD_MUNICIPIO_RESIDENCIA',
       'NO_MUNICIPIO_RESIDENCIA', 'COD_UF_RESIDENCIA', 'UF_RESIDENCIA',
       'IN_ESTUDA_CLASSE_HOSPITALAR', 'COD_ESCOLA', 'COD_MUNICIPIO_ESC',
       'NO_MUNICIPIO_ESC', 'COD_UF_ESC', 'UF_ESC',
       'ID_DEPENDENCIA_ADM_ESC', 'ID_LOCALIZACAO_ESC', 'SIT_FUNC_ESC',
       'IDADE', 'TP_SEXO', 'NACIONALIDADE', 'COD_MUNICIPIO_NASCIMENTO',
       'NO_MUNICIPIO_NASCIMENTO', 'COD_UF_NASCIMENTO', 'UF_NASCIMENTO',
       'ST_CONCLUSAO', 'ANO_CONCLUIU', 'TP_ESCOLA', 'IN_TP_ENSINO',
       'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA',
       'IN_SURDEZ', 'IN_DEFICIENCIA_AUDITIVA', 'IN_SURDO_CEGUEIRA',
       'IN_DEFICIENCIA_FISICA', 'IN_DEFICIENCIA_MENTAL',
       'IN_DEFICIT_ATENCAO', 'IN_DISLEXIA', 'IN_GESTANTE', 'IN_LACTANTE',
       'IN_IDOSO', 'IN_AUTISMO', 'IN_SABATISTA', 'IN_BRAILLE',
       'IN_AMPLIADA_24', 'IN_AMPLIADA_18', 'IN_LEDOR', 'IN_ACESSO',
       'IN_TRANSCRICAO', 'IN_LIBRAS', 'IN_LEITURA_LABIAL

In [29]:
len(['NU_NOTA_LC', 'NU_NOTA_CN', 'NU_NOTA_CH', 'TP_PRESENCA_MT', 'NU_NOTA_MT', 'SG_UF_RESIDENCIA', 'TP_PRESENCA_CH', 
 'NU_IDADE', 'TP_ANO_CONCLUIU', 'TP_PRESENCA_LC', 'TP_ST_CONCLUSAO', 'TP_PRESENCA_CN'])

12

In [30]:
df_2014 = df_2014.rename(columns={'NOTA_LC': 'NU_NOTA_LC', 'NOTA_CN': 'NU_NOTA_CN', 'NOTA_CH': 'NU_NOTA_CH',  'NOTA_MT': 'NU_NOTA_MT',
                        'IN_PRESENCA_MT': 'TP_PRESENCA_MT', 'IN_PRESENCA_CH': 'TP_PRESENCA_CH', 
                        'UF_RESIDENCIA': 'SG_UF_RESIDENCIA', 'IDADE': 'NU_IDADE', 'ANO_CONCLUIU': 'TP_ANO_CONCLUIU', 
                        'IN_PRESENCA_LC': 'TP_PRESENCA_LC', 'ST_CONCLUSAO': 'TP_ST_CONCLUSAO', 'IN_PRESENCA_CN': 'TP_PRESENCA_CN'})

In [31]:
df_2014[cols_used]

,NU_INSCRICAO,NU_ANO,NO_MUNICIPIO_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,140000000001,2014,SAO BERNARDO DO CAMPO,SP,18,F,0,1,1,2013.0,...,A,A,A,A,D,B,A,3,4,4.0
1,140000000002,2014,CONTAGEM,MG,24,M,0,2,4,NaN,...,C,D,D,D,D,A,A,5,4,NaN
2,140000000003,2014,SAO PAULO,SP,19,M,0,2,1,2012.0,...,C,A,A,A,D,A,A,5,3,NaN
3,140000000004,2014,SAO PAULO,SP,18,F,0,1,1,2013.0,...,B,D,D,D,D,A,A,4,4,NaN
4,140000000005,2014,VITORIA DA CONQUISTA,BA,45,M,1,1,1,2004.0,...,B,A,A,D,D,B,A,5,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,140000000996,2014,BRASILIA,DF,26,F,0,3,1,2010.0,...,C,A,A,D,D,A,B,5,3,NaN
996,140000000997,2014,FRANCA,SP,26,M,0,1,1,2007.0,...,C,A,D,D,D,B,A,5,5,NaN
997,140000000998,2014,JATAI,GO,19,M,0,3,1,2013.0,...,B,A,D,D,D,A,C,5,5,NaN
998,140000000999,2014,RIO DE JANEIRO,RJ,17,F,0,3,2,NaN,...,C,A,A,D,D,B,C,5,5,NaN


In [32]:
df_2015 = load_data_year(2015, cols_used)
df_2015.columns.values

array(['NU_INSCRICAO', 'NU_ANO', 'NO_MUNICIPIO_RESIDENCIA',
       'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'TP_PRESENCA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001',
       'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009',
       'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017',
       'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025'],
      dtype=object)